# Coding first examples

In [2]:
# Crear una serie de fibonacci
n = 30
fbci = [0, 1]
for _ in range(2, n):
    fbci.append(fbci[-1] + fbci[-2])
print(fbci)

[0, 1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233, 377, 610, 987, 1597, 2584, 4181, 6765, 10946, 17711, 28657, 46368, 75025, 121393, 196418, 317811, 514229]


In [3]:
for _ in range(20):
    print(_)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
